# Comparison of derived galaxy properties from VAGC and NSA

In [3]:
import numpy as np
import h5py
from pydl.pydlutils.spheregroup import spherematch
from ChangTools.fitstables import mrdfits

import matplotlib.pyplot as plt
from ChangTools.plotting import prettyplot
from ChangTools.plotting import prettycolors
%matplotlib inline 

/usr/local/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [4]:
local_dir = '/Volumes/750GB/hahn/cenque/observations/'

In [ ]:
# VAGC group catalog (all) M_r cut = 18
vagc_file = ''.join(['/Volumes/750GB/hahn/cenque/observations/', 'GroupCat.Mr18.Mass9.4.D360.all.hdf5']) 
f = h5py.File(vagc_file, 'r')
grp = f['data']

# save to dictionary
vagc = {}
for key in grp.keys(): 
    vagc[key] = grp[key].value

# Make combined sample of NSA-2MASS

In [16]:
nsa_data = mrdfits(local_dir+'nsa_v0_1_2.fits')
print len(nsa_data.ra), 'gaalxies'

145155 gaalxies


In [20]:
twomass_data = mrdfits(local_dir+'object_twomass.fits')
print len(twomass_data.ra), 'galaxies'

2506754 galaxies


In [21]:
print nsa_data.ra[:10], nsa_data.dec[:10]
print twomass_data.ra[:10], twomass_data.decl[:10]

[ 146.71420879  146.63173521  147.17644695  147.32950522  146.85676078
  146.19324132  146.00780138  146.59449799  146.61906338  146.7517484 ] [-1.04128157 -0.98835486 -0.35403042  0.02889491 -0.27409464 -0.68939139
 -0.64225922 -0.13314606 -0.43434546 -0.40840274]
[   0.          0.          0.        337.450317  338.114258    0.          0.
  340.849152    0.        341.098816] [ 0.        0.        0.        1.266192  1.271849  0.        0.        1.2714
  0.        1.266147]


In [22]:
m_nsa_2mass, m_2mass_nsa, dd = spherematch(nsa_data.ra, nsa_data.dec, twomass_data.ra, twomass_data.decl, 0.001)

In [23]:
print len(np.unique(m_nsa_2mass)), len(m_nsa_2mass)

64773 64773


In [24]:
nsa = {} 
for key in nsa_data.__dict__.keys(): 
    nsa[key] = getattr(nsa_data, key)

In [26]:
for band in ['j', 'h', 'k']:  # 2MASS bands
    for tag in ['_k20fe', '_ext', '_fe']:
        nsa['2mass_'+band+'_m'+tag] = np.repeat(-999., len(nsa['ra']))
        nsa['2mass_'+band+'_msig'+tag] = np.repeat(-999., len(nsa['ra']))
    
        nsa['2mass_'+band+'_m'+tag][m_nsa_2mass] = getattr(twomass_data, band+'_m'+tag)[m_2mass_nsa]
        nsa['2mass_'+band+'_msig'+tag][m_nsa_2mass] = getattr(twomass_data, band+'_msig'+tag)[m_2mass_nsa]

In [27]:
print nsa['z'].min(), nsa['z'].max()

-0.00333485 0.0550015
